# Spelling Bee Challenge with RNN

- Uses seq2seq model

[reference](https://github.com/mikesj-public/rnn_spelling_bee/blob/master/spelling_bee_RNN.ipynb)

In [14]:
# preprocessing data
START_LINE = 126
END_LINE = 133905
FILENAME = 'data/cmudict-0.7b'
data = open(FILENAME, 'r', encoding='utf-8', errors='ignore').read()

'\n'